# Matched filtering
This notebook uses the second example signal from the Inverse filtering notebook, and applies a matched filter instead of inverse filtering.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import scipy.signal as sps
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified.

Parameter | Meaning
--------- | -------
<code>number_of_samples</code>| length of the time record to evaluate
<code>X</code> | an example waveform that we wish to detect in a data signal

In [ ]:
number_of_samples = 60
X = [1, -0.7, -0.3, -0.534, 0.6305, -0.23375]

### Plotting Function

In [ ]:
def plot_stem(x, y, title, name):
    """
       Create a stem plot of sample.
       
       INPUT:
            x       (array-like): The x-positions of the stems. 
            y       (array-like): The y-values of the stem heads.
            title       (string): The title of the figure.
            name        (string): The name used to save figure.
    """
    plt.figure(figsize = (12, 6))
    plt.rcParams.update({'font.size': 16})
    
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection=True)
    plt.setp(baseLines, color = 'black', linewidth=1) 
    markerLines.set_markerfacecolor('none')
    
    plt.xlabel('Sample')
    plt.xlim([0, x[-1]])
    plt.ylabel('Output')
    plt.title(title)
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

### Generate matched filter and signal
A matched filter is a time reversal of the signal that we are looking for.  As in the Inverse filter notebook, we create a signal that consists of noise, and the signal we are looking for added to this noise.

In [ ]:
# Create matched filter
matched_filter = np.flip(X, 0)

# Time index used for plotting
n = np.arange(0, number_of_samples)

# Generate noisy signal to be analysed
noise = np.random.randn(number_of_samples) * 0.1
Y = noise.copy()
Y[30:30+len(X)] = Y[30:30+len(X)] + X

### Apply the matched filter
We apply the matched filter, defined above, as we would any other filter.  The output of the filter has the largest peak at the timepoint corresponding to the time at which the signal we are looking for appears.

In [ ]:
V = sps.lfilter(matched_filter, [1], Y)

title = 'Matched filter output'
name = 'Matched_filter.pdf'
plot_stem(n, V, title, name)

### Non sample spaced delays
Now we want to investigate delaying the signal by a non-integer amount.  This can be done using interpolation.  We first of all increase the sampling rate by a factor of 10, then select a set of samples offset by 0.3 from the original samples (i.e. 3 samples at the new sampling rate).  We can then simply subsample these data (there is no need for a decimation filter) to obtain the fractionally shifted samples that we want.

In [ ]:
temp = np.zeros(number_of_samples)
temp[30:30+len(X)] = X

# Resample by a factor of 10
highrate_temp = sps.resample_poly(temp, up=10, down=1)

# Create an index into the resampled array offset by 3 samples
step = np.arange(3, number_of_samples*10, 10)

# Extract samples at these timepoints into a new array
fractional_spaced = list(highrate_temp[step])

# Create noisy version of this new signal
Y_frac = noise + fractional_spaced

As before, we apply the matched filter and plot it

In [ ]:
U = sps.lfilter(matched_filter, [1], Y_frac)

title = 'Matched filter output'
name = 'Matched_filter_fractional_spaced.pdf'
plot_stem(n, U, title, name)

### Upsampling for detection
The peak of the output when the input is fractionally mis-aligned to the signal is lower than it was in the first plot on this notebook.  We can use interpolation and matched filtering on this interpolated signal to obtain a higher peak, and also detect the time that the signal appears with better accuracy.

In [ ]:
# Upsample the received signal
YU = sps.resample_poly(Y_frac, up=20, down=1)
nu = np.arange(0, number_of_samples, 1/20)
temp = np.zeros(20)
temp[0] = 1

# Generate a matched filter at this higher sampling rate
G = np.kron(matched_filter, temp)

# Apply it to the upsampled signal
UU = sps.lfilter(G, [1], YU)

# and plot it
plt.figure(figsize = (12, 6))
plt.plot(nu, UU)
plt.xlim([0, nu[-1]])
plt.title('Matched filter output', fontsize = 16)
if not is_jupyter():
    plt.tight_layout()
    plt.savefig('Matched_filter_fractional_spaced_upsampled.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.